# Introduction to Machine Learning (CSCI-UA.473)

## Homework 4: Long-Short Term Memory Networks and Ensemble of Neural Network Models
### Due: November 18th, 2021 at 11:59PM


### Name: Jerry Jia
### Email: tj1043

This homework has two parts. In the first part you will implement the long-short term memory networks and train them on a specific sequence modeling task. In the second part you will build an ensemble of neural networks for a specific problem and conduct some error analysis.

## Question P1: Long-Short Term Memory Networks (35 Points Total)

In this problem you will implement a long-short term memory (LSTM) models from scratch and train them for the language modeling task. You will also do some error analysis and ablation studies on the learnt model. 

**At no point are you allowed to use PyTorch's nn.LSTM function in the code. Remember the goal of this assignment is to build the LSTM from scratch.**

### Define some helper functions to handle the text data (do nothing here)

In [1]:
import torch
import os

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self):
        return len(self.word2idx)


class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words: 
                    self.dictionary.add_word(word)  
        
        # Tokenize the file content
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        return ids.view(batch_size, -1)
 

### Define some hyper-parameters of the model (do nothing here.. yet)
We will now define some hyper-parameters to be used with the model. Later on in the assignment you will be experimenting with some of these. 

In [2]:
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
embed_size = 32         # size of the input feature vector representing each word
hidden_size = 32        # number of hidden units in the LSTM cell
num_epochs = 1          # number of epochs for which you will train your model    
num_samples = 200       # number of words to be sampled
batch_size = 20         # the size of your mini-batch
seq_length = 30         # the size of the BPTT window
learning_rate = 0.002   # learning rate of the model

device

device(type='cuda')

### Load the sequential data (do nothing here)

We will use the Penn Tree Bank dataset for the purpose of this exercise. You need to download by running the command `wget https://data.deepai.org/ptbdataset.zip`, unzip it, and store it in the directory `./data/ptb`. We will only use the files `ptb.train.txt` and `ptb.test.txt`. 

In [3]:
# Load the dataset 
corpus = Corpus()
ids = corpus.get_data('./data/ptb/ptb.train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

test_corpus = Corpus()
test_ids = test_corpus.get_data('./data/ptb/ptb.test.txt', batch_size)
test_vocab_size =  len(test_corpus.dictionary)
test_num_batches = test_ids.size(1) // seq_length

print(num_batches)
print(test_num_batches)

1549
137


### Implement the Long-Short Term Memory (LSTM) unit (20 Points)

You will be implementing a single unit of the LSTM and also write its forward propagation algorithm. The single LSTM unit performs the following operations: 

$$
\begin{align}
i_t & = \sigma (W_{ii} x_t + b_{ii} + W_{hi} h_{t-1} + b_{hi}) \\
f_t & = \sigma (W_{if} x_t + b_{if} + W_{hf} h_{t-1} + b_{hf}) \\
g_t & = \sigma (W_{ig} x_t + b_{ig} + W_{hg} h_{t-1} + b_{hg}) \\
o_t & = \sigma (W_{io} x_t + b_{io} + W_{ho} h_{t-1} + b_{ho}) \\
c_t & = f_t \odot c_{t-1} + i_t \odot g_t \\
h_t & = o_t \odot \tanh(c_t)
\end{align}
$$

where $h_t$ is the hidden state at time $t$, $c_t$ is called the memory cell state at time $t$, $x_t$ is the input at time $t$, $h_{t-1}$ is the hidden state of the layer at time $t-1$ or the initial hidden state at time $0$, and $i_t$, $f_t$, $g_t$, $o_t$, are the input, forget, cell, and output gates, respectively. $\sigma$ is the sigmoid function, and $\odot$ is the Hadamard (element-wise) product of two vectors. Finally $\{W_{ii}, b_{ii}, W_{hi}, b_{hi}\}$, $\{W_{if}, b_{if}, W_{hf}, b_{hf}\}$, $\{W_{ig}, b_{ig}, W_{hg}, b_{hg}\}$, and $\{W_{io}, b_{io}, W_{ho}, b_{ho}\}$ are the learnable weights and biases for computing the input, forget, cell, and output gates respectively.

At each time step the LSTM takes as input the previous hidden state, the previous memory cell state and the embedding (features) associated with the current word and generates the new hidden states and the prediction of the next word. 

**Hint: Note that the recurrsion is around two variables, namely, $h_t$ and $c_t$. Hence you will need to keep track of two previous hidden states.**

In [4]:
# myLSTM based language model
class myLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        """
        Input: 
            vocab_size: the number of unique words in your dataset 
            embed_size: the size of the feature vector assiciated with each word
            hidden_size: the size of the hidden state features
        """
        super(myLSTM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size) # features associated with the words in your vocabulary
        
        # rest of your LSTM model code goes here
        
    def forward(self, x, h):
        """
        Perform forward propagation over a single unit composed of an LSTM unit followed by a linear layer to 
        generate the input activations for the softmax function. 
    
        Input:
            x: the current input (indices into the vocabulary)
            h: the previous hidden states

        Output:
            out: the output of the unit (which are the input activations of the softmax layer to predict the next word)
            (h, c): the current hidden states composed of the hidden state and the memory cell state
        """
        
        # your code goes here
        return out, (h, c)

model = myLSTM(vocab_size, embed_size, hidden_size)


# Loss and optimizer
criterion = ?     # the loss function

# we will use the Adam optimizer for faster and easier convergence
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

SyntaxError: invalid syntax (1276616913.py, line 36)

?? Use cross entropy loss function from pytorch

### Training loop for the model (7.5 Points)

In [ ]:
# Truncated backpropagation
def detach(states):
    # Detach from pytorch computational graph
    return [state.detach() for state in states]


# Train the model
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(1, batch_size, hidden_size).to(device),
              torch.zeros(1, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Forward pass
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        states = detach(states)
        
        ## the code for the rest of the forward pass goes here ##
        
        
        # Backward pass 
        
        ## code for backward pass goes here ##

        # we clip the gradients to ensure that they remain bounded. This is a hack! 
        clip_grad_norm_(model.parameters(), 0.5)
        
        # Optimize        
        
        ## code for optimization goes here ##

        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

### Testing loop for the model (7.5 Points)

In [ ]:
import math

# Test the model
states = (torch.zeros(1, batch_size, hidden_size).to(device),
              torch.zeros(1, batch_size, hidden_size).to(device))
test_loss = 0.
with torch.no_grad():
    for i in range(0, test_ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = test_ids[:, i:i+seq_length].to(device)
        targets = test_ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Forward pass
        states = detach(states)
        
        ## code for forward pass goes here ##

test_loss = test_loss / test_num_batches
print('test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))


### Generating text using the trained model

In [ ]:
# Generate texts using trained model
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # Set intial hidden ane cell states
        state = (torch.zeros(1, 1, hidden_size).to(device),
                 torch.zeros(1, 1, hidden_size).to(device))

        # Select one word id randomly
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

        for i in range(num_samples):
            # Forward propagate RNN 
            output, state = model(input, state)

            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()

            # Fill input with sampled word id for the next time step
            input.fill_(word_id)

            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, num_samples, 'sample.txt'))

**As part of your solution please submit the sample.txt file as well. Name it as \<your-netid>_sample_hw4.txt and include it in your top level zip file**

## Question P2: Ensemble of Neural Networks (35 Points Total)
In this part of the assignment, you will train an ensemble of neural networks for classifying hand written digits in the MNIST dataset. You will take the code provided in the following cell and extend it to build an ensemble of single hidden layer MLP as per the specifications provided in the questions below. 

For the data, you need to download the MNIST dataset from ``http://yann.lecun.com/exdb/mnist/`` and place the files in the directory ``./data/MNIST``. 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy


# load the MNIST dataset
input_size  = 28*28   # images are 28x28 pixels
output_size = 10      # there are 10 classes

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/MNIST', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/MNIST', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True)


# define the multi-layer perceptron model
class FC2Layer(nn.Module):
    def __init__(self, input_size, n_hidden, output_size):
        super(FC2Layer, self).__init__()
        self.input_size = input_size
        self.network = nn.Sequential(
            nn.Linear(input_size, n_hidden), 
            nn.ReLU(), 
            nn.Linear(n_hidden, n_hidden), 
            nn.ReLU(), 
            nn.Linear(n_hidden, output_size), 
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = x.view(-1, self.input_size)
        return self.network(x)
    

# define the training loop
def train(epoch, model):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        optimizer.zero_grad()
        # forward pass through the model
        output = model(data)
        # forward pass through the cross-entropy loss function
        loss = F.nll_loss(output, target)
        # backward pass through the cross-entropy loss function and the model
        loss.backward()
        
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


# define the testing loop
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss                                                               
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    accuracy_list = []
    accuracy_list.append(accuracy)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

    
# build the model and execute the training and testing

# initialize some hyper-paramters 
n_hidden = 4 # number of hidden units
learning_rate = 0.01
moment = 0.5
nepochs = 1

# build the actual model
model_fnn = FC2Layer(input_size, n_hidden, output_size)
#model_fnn.to(device)
# initialize the optimizer
optimizer = optim.SGD(model_fnn.parameters(), lr=learning_rate, momentum=moment)
print('Number of parameters: {}'.format(get_n_params(model_fnn)))

# train the model for one epoch
for epoch in range(0, nepochs):
    train(epoch, model_fnn)
    test(model_fnn)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



NameError: name 'get_n_params' is not defined

### Question P2.a: Ensemble of networks of same size but initialized with a different seed (20 Points)
Extend the above code to create an ensemble of `k` single hidden layer MLP models, each initialized with a different random seed. Train each model inside the ensemble for exactly `1` epoch and report the performance of the individual models and the ensemble model on the test set. For the purpose of ensembling, you can take a majority vote of the model outputs. 

At a high level you need to perform the following tasks:
1. Initialize `k` models with different random seed (for some value of k): (**5 points**)
2. Extend the train() function to train individual models inside the ensemble: (**5 points**)
3. Extend the test() function to estimate the accuracy of the individual models and the ensemble model: (**5 points**)
4. Repeat the above process with different values of `k`. For this exercise use $k = \{1, 2, 4, 8, 16, 32 \}$, and plot a graph with `k` on the x-axis and ensemble model performance on the y-axis. (**5 points**)

### Question P2.b: Bagged networks  (15 Points)
Extend the above code to create a bag of `k` single hidden layer MLP models with each with `4` hidden units. Compare the performance of bags of different sizes `k`. In particular, compare the performance of `5` bages, with sizes $k = \{2, 4, 8, 16, 32\}$ respectively. To prevent additional sources of randomness, all the models within all the bags should be initialized to the same random seed. Train each model inside each bag for exactly `1` epoch. Report the final performance for each bag on the test set and compare it against each other. For the purpose of ensembling, you can take a majority vote of the model outputs. 

At a high level you need to perform the following tasks:
1. Create `k=5` bags of the training set of sizes $k = \{2, 4, 8, 16, 32\}$. (**5 points**)
2. Use the same extension to the train() function as in P2.a to train individual models in each bag: (**2.5 points**)
3. Use the same extension to the test() function as in P2.a to estimate the accuracy for each bag: (**2.5 points**)
4. Plot a graph with bag_size on the x-axis and bag performance on the y-axis. (**5 points**)